# Summary

* Train the model: `rf.boston <- randomForest(medv ~ . - medv, data = Boston, subset = train)`
    * The "Mean of squared residuals" is the out of bag residuals
    * mtry is the # of predictors used in each split
    * mse is an vector, corresponds to the number of tree
* Make prediction
    * pred <- predict(fit, Boston[-train,])
    
## Boosting

# Random forest 

In [ ]:
require(randomForest)
require(MASS)

In [ ]:
set.seed(101)

In [ ]:
dim(Boston)

In [ ]:
train <- sample(1:nrow(Boston), 300)

In [ ]:
length(train)

In [ ]:
rf.boston <- randomForest(medv ~ ., data = Boston, subset = train)
str(rf.boston)

In [ ]:
oob.err = double(13)
test.err = double(13)
for (mtry in 1:13) {
    fit <- randomForest(medv ~ ., data = Boston, subset = train, mtry = mtry, ntree = 400)
    oob.err[mtry] <- fit$mse[400]
    pred <- predict(fit, Boston[-train,])
    test.err[mtry] <- with(Boston[-train,], mean((pred-medv)^2))
    cat(mtry, "")
}

In [ ]:
matplot(1:mtry, cbind(test.err, oob.err), pch = 19, col = c("red", "blue"), type = "b")
legend("topright", legend = c("Test", "OOB"), pch = 19, col = c("red", "blue"))